## Devices 

> Interfaces to the various smart home providers

In [ ]:
# | default_exp devices
# | export
import os

from typing import Any, Optional
from pydantic import BaseModel
from tuya_iot import TuyaOpenAPI
from switchbot_client import SwitchBotClient

## Base Models

In [ ]:
# | export
class Device:
    "Base class for all smart home devices"


class AirPurifier:
    "Air Purifier device"


class Fan(Device):
    "A Fan device"


class CeilingFan(Device):
    "A Fan that is attached to the ceiling"


class TuyaDevice(BaseModel):
    """
    A device controlled by the Tuya Cloud.
    """

    client_id: str
    client_secret: str
    username: str
    password: str
    country_code: str
    endpoint: str = "https://openapi.tuyaus.com"
    appschema: str = "smartLife"
    client: Optional[Any]

    def connect(self):
        self.client = TuyaOpenAPI(self.endpoint, self.client_id, self.client_secret)
        self.client.connect(
            self.username, self.password, self.country_code, self.appschema
        )


class TuyaSceneDevice(TuyaDevice):
    """
    A device controlled via scenes in the Tuya Cloud.
    """

    home_id: str

    def trigger_scene(self, id: str):
        self.client.post(f"/v1.0/homes/{self.home_id}/scenes/{id}/trigger")

    def trigger(self, action: str):
        self.trigger_scene(getattr(self, f"{action}_scene_id"))

In [ ]:
from nbdev.showdoc import show_doc

## Air Purifier

We create a `TuyaAirPurifier` that we can use to turn on and off the air purifier.

In [ ]:
# | export
class TuyaAirPurifier(TuyaDevice, AirPurifier):
    """Purifiers controlled by a Tuya Smart Switch"""

    device_id: str

    def switch(self, on: bool = True):
        self.client.post(
            f"/v1.0/devices/{self.device_id}/commands",
            {"commands": [{"code": "switch_1", "value": on}]},
        )

    def turn_on(self):
        self.switch(True)

    def turn_off(self):
        self.switch(False)

## Ceiling Fans 

I have a three ceiling fans in the home controlled through RF frequencies.

In [ ]:
# | export
class DaikoCeilingFan1(TuyaSceneDevice, CeilingFan):
    light_scene_id: str
    fan_scene_id: str
    fan_1_scene_id: str
    fan_2_scene_id: str
    fan_3_scene_id: str

In [ ]:
# | export
living_area_ceiling_fan = DaikoCeilingFan1(
    home_id=os.getenv("TUYA_HOME_ID"),
    client_id=os.getenv("TUYA_CLIENT_ID"),
    client_secret=os.getenv("TUYA_CLIENT_SECRET"),
    username=os.getenv("SMART_LIFE_USERNAME"),
    password=os.getenv("SMART_LIFE_PASSWORD"),
    country_code="PH",
    light_scene_id="AniQzKuX7A98n9gn",
    fan_scene_id="tmAFQYbx1oynJcG6",
    fan_1_scene_id="cXSEy5ALHoR4LT5H",
    fan_2_scene_id="fTwRU120YlR0nWOo",
    fan_3_scene_id="hOCfvtwpp1urauns",
)

In [ ]:
# | export
class DaikoCeilingFan2(TuyaSceneDevice, CeilingFan):
    light_scene_id: str
    fan_off_scene_id: str
    fan_1_scene_id: str
    fan_2_scene_id: str
    fan_3_scene_id: str

In [ ]:
# | export
master_bedroom_ceiling_fan = DaikoCeilingFan2(
    home_id=os.getenv("TUYA_HOME_ID"),
    client_id=os.getenv("TUYA_CLIENT_ID"),
    client_secret=os.getenv("TUYA_CLIENT_SECRET"),
    username=os.getenv("SMART_LIFE_USERNAME"),
    password=os.getenv("SMART_LIFE_PASSWORD"),
    country_code="PH",
    light_scene_id="ooGJVKFOQ586ZPDb",
    fan_off_scene_id="tIOVX91BOrpgH86A",
    fan_1_scene_id="JmJpPLVlqhURktzX",
    fan_2_scene_id="Y3tAAqXdEGfarxje",
    fan_3_scene_id="rS34nHG6BRE4ddza",
)

In [ ]:
# | export
common_bedroom_ceiling_fan = DaikoCeilingFan2(
    home_id=os.getenv("TUYA_HOME_ID"),
    client_id=os.getenv("TUYA_CLIENT_ID"),
    client_secret=os.getenv("TUYA_CLIENT_SECRET"),
    username=os.getenv("SMART_LIFE_USERNAME"),
    password=os.getenv("SMART_LIFE_PASSWORD"),
    country_code="PH",
    light_scene_id="RQB0qD2ec5AqPmT3",
    fan_off_scene_id="RWA1GutcWMWTuaPu",
    fan_1_scene_id="ec4UBLRrVukAm5rG",
    fan_2_scene_id="bHsXR52LReWvhxSL",
    fan_3_scene_id="mg8wiQ4u9WTMy6BM",
)

## SwitchBots 

The Ceiling Fans are attached to switches controlled by Switchbot, so in order to control it, I'm going to need to also control the Switchbot.

In [ ]:
import requests
import hashlib
import hmac
import base64
import time
import uuid
import os
import json

from urllib.parse import urljoin


def switchbot_device_list():
    """
    Existing Python API clients do not work with the DIY Fan Controller and cause an
    error upon device listing. This function is designed to return that list for use
    in the other API clients.
    """
    timestamp = int(round(time.time() * 1000))
    opentoken = os.getenv("SWITCHBOT_OPEN_TOKEN")
    secretkey = os.getenv("SWITCHBOT_SECRET_KEY")
    signature = bytes(f"{opentoken}{timestamp}", "utf-8")
    secretkey = bytes(secretkey, "utf-8")
    signature = base64.b64encode(
        hmac.new(secretkey, msg=signature, digestmod=hashlib.sha256).digest()
    )

    switchbot_api_host = "https://api.switch-bot.com"
    response = requests.get(
        url=urljoin(switchbot_api_host, "/v1.1/devices"),
        headers={
            "Authorization": opentoken,
            "sign": signature,
            "nonce": "",
            "t": str(timestamp),
        },
    )
    return json.loads(response.content)["body"]["deviceList"]

In [ ]:
# | export


class SwitchBot(BaseModel):
    "A Switchbot controlling a light switch"
    device_id: str
    device: Optional[Any]
    client: Optional[Any]

    def connect(self):
        self.client = SwitchBotClient()
        self.device = self.client.device(self.device_id)
        return self

    def turn_on(self):
        self.device.turn_on()

    def turn_off(self):
        self.device.turn_off()

In [ ]:
master_bedroom_ceiling_fan_switch = SwitchBot(device_id="E307787C88C3")

In [ ]:
common_bedroom_ceiling_fan_switch = SwitchBot(device_id="F326616FB7D2")